## A3:  Subqueries

```
Your name: _Dillon Wu_________________


AndrewID: ___dillonw________________

Queries that work:
All of them

Queries that don't:
```None of them


**Optional**:  If the table `Medalists` is not already created, you can recreate it using the instructions from assignment 2:

```
% psql -d postgres -U isdb -f a2_load.sql
```


### While some of these queries could be written using a JOIN, for this assignment you should not use a JOIN.  Rather, as needed, express your solution using sub-queries

In [1]:
%load_ext sql
%sql postgres://isdb@localhost/postgres

'Connected: isdb@postgres'

### The Schema of the Medalists table

In [2]:
%sql select * from medalists limit 0

 * postgres://isdb@localhost/postgres
0 rows affected.


edition,athlete,noc,gender,event,medal,season


**[Q 01] In a2 we saw that though the official Winter Olympic games started in 1924, winter sports in Olympic games started in 1908. In descending order by total medal count, list the countries which won medals in the very first winter sports.**

https://en.wikipedia.org/wiki/Figure_skating_at_the_1908_Summer_Olympics 

**You are not allowed to hard code the value of 1908 in your query.  Rather, you need to write a sub-query that will first determine the year in which winter sports were first part of the olympics (i.e., 1908) and then use this sub-query in your main query to determine the final answer.**

In [3]:
%%sql
--solution
SELECT m.noc, count(m.medal)
  FROM medalists as m
 WHERE m.edition = (SELECT min(edition)
                      FROM medalists
                     WHERE season = 'Winter')
   AND season = 'Winter'
 GROUP BY m.noc
 ORDER BY count(m.medal) desc; 






 * postgres://isdb@localhost/postgres
4 rows affected.


noc,count
GBR,7
GER,4
SWE,3
RU1,1


In a2 we saw that including 2004, eight men athletes have won medals more than once in the 100m.  Seven women athletes have also won medals more than once with in the 100m, with OTTEY-PAGE, Merlene winning thrice!
https://en.wikipedia.org/wiki/Merlene_Ottey 
```
+---------------------------------+-------+---+
|             athlete             | event | c |
+---------------------------------+-------+---+
|       OTTEY-PAGE, Merlene       |  100m | 3 |
|     WALASIEWICZ, Stanislawa     |  100m | 2 |
|           TYUS, Wyomia          |  100m | 2 |
|           DEVERS, Gail          |  100m | 2 |
|         ASHFORD, Evelyn         |  100m | 2 |
| STRICKLAND-DE LA HUNTY, Shirley |  100m | 2 |
|         STECHER, Renate         |  100m | 2 |
+---------------------------------+-------+---+
```

**[Q 02] Using a sub-query, write a query that will chronologically list information about each woman athlete who won a medal in the 100m (year, country, name, medal). **

In [4]:
%%sql
--solution 
SELECT m.edition, m.noc, m.athlete, m.medal
  FROM medalists AS m
 WHERE m.athlete IN (SELECT athlete
                       FROM medalists
                      WHERE event = '100m'
                        AND gender = 'Women'
                      GROUP BY athlete
                     HAVING count(medal) >= 2)
   AND m.event = '100m'
 ORDER by m.edition asc;











 * postgres://isdb@localhost/postgres
15 rows affected.


edition,noc,athlete,medal
1932,POL,"WALASIEWICZ, Stanislawa",Gold
1936,POL,"WALASIEWICZ, Stanislawa",Silver
1948,AUS,"STRICKLAND-DE LA HUNTY, Shirley",Bronze
1952,AUS,"STRICKLAND-DE LA HUNTY, Shirley",Bronze
1964,USA,"TYUS, Wyomia",Gold
1968,USA,"TYUS, Wyomia",Gold
1972,GDR,"STECHER, Renate",Gold
1976,GDR,"STECHER, Renate",Silver
1984,USA,"ASHFORD, Evelyn",Gold
1984,JAM,"OTTEY-PAGE, Merlene",Bronze


**[Q 03] First, write a query to produce a table of unique NOCs**
https://en.wikipedia.org/wiki/National_Olympic_Committee
Call the table `nocs` if you count its length you should get 139.  

As always, for ease of re-execution, precede a `CREATE TABLE ...` with a `DROP TABLE IF EXISTS ...`

 

In [5]:
%%sql
--solution
DROP TABLE IF EXISTS nocs;
CREATE TABLE nocs AS
    SELECT DISTINCT noc
      FROM medalists;

 * postgres://isdb@localhost/postgres
Done.
139 rows affected.


[]

**[Q 04] Write a query to produce a count of the number of Gold medals each country has won.  Call this table `Gold_Medals`.** 

The sub-query you will use is placed in the SELECT clause.  The outline of your solution will be

```
CREATE TABLE Gold_Medals AS
   SELECT _______, (SELECT __________) as count
   ...
 ```

In [6]:
%%sql
-- solution
DROP TABLE IF EXISTS Gold_Medals;
CREATE TABLE Gold_Medals AS
   SELECT m.noc, (SELECT count(m2.medal)
                    FROM Medalists AS m2
                   WHERE m2.medal = 'Gold'
                     AND m2.noc = m.noc 
               )AS count
     FROM nocs AS m;


 * postgres://isdb@localhost/postgres
Done.
139 rows affected.


[]

In [7]:
%sql select count(*) from Gold_Medals

 * postgres://isdb@localhost/postgres
1 rows affected.


count
139


Your table should mirror the below

In [8]:
%%sql
SELECT * 
 FROM Gold_Medals
ORDER BY count desc
LIMIT 5;

 * postgres://isdb@localhost/postgres
5 rows affected.


noc,count
USA,2187
URS,1003
GBR,514
ITA,509
GER,480


**[Q 05] Which countries have not won a gold medal at all? Just give the NOC and count (to verify).  Sort the table alphabetically and list the first 10**

In [9]:
%%sql 
-- solution

SELECT noc, count
  FROM Gold_Medals
 WHERE count = '0'
 ORDER BY noc ASC
 LIMIT 10;








 * postgres://isdb@localhost/postgres
10 rows affected.


noc,count
AFG,0
AHO,0
BAR,0
BER,0
BOH,0
BWI,0
CIV,0
DJI,0
ERI,0
GHA,0


**[Q 06] Which countries have won the most number of medals without winning a gold?**

Hint: In the previous query you found all the NOCs which haven't won any gold medal.  Using this as a subquery, filter and group medalists.  You will need to use the 'in' operator.  read about it in https://www.postgresql.org/docs/current/static/functions-subquery.html#FUNCTIONS-SUBQUERY-IN 

In [10]:
%%sql
-- solution

SELECT m.noc, count(m.medal) AS "Total Number of Medals"
  FROM Medalists AS m
 WHERE m.noc NOT IN (SELECT noc
                       FROM Gold_Medals
                      WHERE count >= 1)
 GROUP BY m.noc
 ORDER BY count(m.medal) desc
 LIMIT 5;








 * postgres://isdb@localhost/postgres
5 rows affected.


noc,Total Number of Medals
PAR,17
ISL,17
GHA,16
SRB,15
SCG,14


**[Q 07] Which countries have won one and only 1 gold?**

In [11]:
%%sql
-- solution

SELECT g.noc
  FROM Gold_Medals AS g
 WHERE g.noc IN (SELECT noc
                   FROM Medalists
                  GROUP BY noc
                 HAVING count(medal) = 1)
   AND count = 1;










 * postgres://isdb@localhost/postgres
2 rows affected.


noc
UAE
BDI
